## This code finds the closest H to the OH from its nearest water 

In [13]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

In [14]:
#path = "C:/Users/Sachith/Desktop/work_oh/test_with_temp_n_oh/one_oh2/500k/thermalization1"
#path = "C:/Users/Sachith/Desktop/work_oh/different_water_density/water40/md26"

In [15]:
#f=open(path +'/input_data.dat','r')
f=open('input_data.dat','r')
lines=f.readlines()
for line in lines[0:]:
    data=line.split()
    if data[0]=='nsteps:':
        nsteps = int(data[1])
    if data[0]=='natoms:':
        natoms = int(data[1])
    if data[0]=='nonmvat:':
        nonmvat = int(data[1])
    if data[0]=='nmovingatoms:':
        nmovingatoms = int(data[1])
    if data[0]=='MDRestartFrequency:':
        MDRestartFrequency = int(data[1])
    if data[0]=='dt:':
        dt = float(data[1])
    if data[0]=='noh:':
        noh = int(data[1])
    if data[0]=='nwater:':
        nwater = int(data[1])
    if data[0]=='nox:':
        nox = int(data[1])
    if data[0]=='nhy:':
        nhy = int(data[1])
    if data[0]=='xbox:':
        xbox = float(data[1])
    if data[0]=='ybox:':
        ybox = float(data[1])
    if data[0]=='zbox:':
        zbox = float(data[1])
        
nsteps = int((nsteps/MDRestartFrequency)+1)-1
nstep = nsteps
        
print('nsteps:','  ',nstep )
print('natoms:','  ',natoms)
print('nonmvat:','  ',nonmvat)
print('nmovingatoms:','  ',nmovingatoms)
print('MDRestartFrequency:','  ',MDRestartFrequency)
print('dt:','  ',dt)
print('noh:','  ',noh)
print('nwater:','  ',nwater)
print('nox:','  ',nox)
print('nhy:','  ',nhy)
print('xbox:','  ',xbox)
print('ybox:','  ',ybox)
print('zbox:','  ',zbox)

nsteps:    970
natoms:    323
nonmvat:    261
nmovingatoms:    62
MDRestartFrequency:    10
dt:    1.0
noh:    1
nwater:    20
nox:    21
nhy:    41
xbox:    12.664
ybox:    13.3552
zbox:    40.0


In [16]:
ox = np.empty(nox,dtype=int)
hy = np.empty(nhy+1,dtype=int)


mm = 0
nn = 1
while mm<(nox):
    ox[mm] = nn+nonmvat
    mm = mm+1
    nn = nn+3

mm = 0
nn =1
while mm<(nhy):
    hy[mm] = nn+nonmvat+1
    hy[mm+1] = nn+nonmvat+2
    mm = mm+2
    nn = nn+3
hy = hy[:-1]


print(ox)
print(hy)

[262 265 268 271 274 277 280 283 286 289 292 295 298 301 304 307 310 313
 316 319 322]
[263 264 266 267 269 270 272 273 275 276 278 279 281 282 284 285 287 288
 290 291 293 294 296 297 299 300 302 303 305 306 308 309 311 312 314 315
 317 318 320 321 323]


In [17]:
msd_of_oh = np.empty(nstep)
msd_of_ohx = np.empty(nstep)
msd_of_ohy = np.empty(nstep)
msd_of_ohz = np.empty(nstep)
oh_index = np.empty(nstep,dtype=int)
xoh = np.empty(nstep)
yoh = np.empty(nstep)
zoh = np.empty(nstep)
atom= ['Non']*(natoms)

clos_O_index = np.empty(nstep,dtype=int)

x_ox = np.empty(nstep*nox).reshape(nox,nstep)
y_ox = np.empty(nstep*nox).reshape(nox,nstep)
z_ox = np.empty(nstep*nox).reshape(nox,nstep)

x_hy = np.empty(nstep*nhy).reshape(nhy,nstep)
y_hy = np.empty(nstep*nhy).reshape(nhy,nstep)
z_hy = np.empty(nstep*nhy).reshape(nhy,nstep)

xv_ox = np.empty(nstep*nox).reshape(nox,nstep)
yv_ox = np.empty(nstep*nox).reshape(nox,nstep)
zv_ox = np.empty(nstep*nox).reshape(nox,nstep)

xv_hy = np.empty(nstep*nhy).reshape(nhy,nstep)
yv_hy = np.empty(nstep*nhy).reshape(nhy,nstep)
zv_hy = np.empty(nstep*nhy).reshape(nhy,nstep)

In [18]:
jj=0
kk=0
#f=open(path +'/geo_end.xyz','r')
f=open('geo_end.xyz','r')
lines=f.readlines()
while jj < nstep:
    io=0
    ih=0
    for line in lines[kk+nonmvat+2:(kk+natoms+2)]:
        data=line.split()
        if len(data)>3:
            if data[0]=='O':
                x_ox[io][jj]=float(data[1])
                y_ox[io][jj]=float(data[2])
                z_ox[io][jj]=float(data[3])
                xv_ox[io][jj]=float(data[5])
                yv_ox[io][jj]=float(data[6])
                zv_ox[io][jj]=float(data[7])
                #print('O=',xv_ox[io][jj],yv_ox[io][jj],zv_ox[io][jj])
                io = io+1
            elif data[0]=='H':
                x_hy[ih][jj]=float(data[1])
                y_hy[ih][jj]=float(data[2])
                z_hy[ih][jj]=float(data[3])
                xv_hy[ih][jj]=float(data[5])
                yv_hy[ih][jj]=float(data[6])
                zv_hy[ih][jj]=float(data[7])
                #print('H=',xv_hy[io][jj],yv_hy[io][jj],zv_hy[io][jj])
                ih = ih+1
    kk=kk+natoms+2
    jj=jj+1
f.close()

In [19]:
jj = 0
#f=open(path +'/oh_to_o_dis.dat','r')
f=open('oh_to_o_dis.dat','r')
lines=f.readlines()
for index, line in enumerate(lines[:]):
    data=line.split()
    oh_index[jj] = int(data[1])
    clos_O_index[jj] = int(data[2])
    #print(data,oh_index[jj],clos_O_index[jj])
    jj=jj+1
    
f.close()

In [20]:
x_ox_box = np.empty(nstep*nox).reshape(nox,nstep)
y_ox_box = np.empty(nstep*nox).reshape(nox,nstep)
z_ox_box = np.empty(nstep*nox).reshape(nox,nstep)
x_hy_box = np.empty(nstep*nhy).reshape(nhy,nstep)
y_hy_box = np.empty(nstep*nhy).reshape(nhy,nstep)
z_hy_box = np.empty(nstep*nhy).reshape(nhy,nstep)

x_ox_box = x_ox.copy()
y_ox_box = y_ox.copy()
z_ox_box = z_ox.copy()
x_hy_box = x_hy.copy()
y_hy_box = y_hy.copy()
z_hy_box = z_hy.copy()

In [21]:
# move atoms in to the box.
ijk = 1
while ijk < 2:
    print(ijk)
    ijk = ijk+1
    jj=0
    while jj < nstep:
        ii=0
        while ii < nox:
            if x_ox_box[ii][jj]>(xbox/2):
                x_ox_box[ii][jj]=x_ox_box[ii][jj]-xbox
                ijk = 1
            if x_ox_box[ii][jj]<(-xbox/2):
                x_ox_box[ii][jj]=x_ox_box[ii][jj]+xbox
                ijk = 1
            
            if y_ox_box[ii][jj]>(ybox/2):
                y_ox_box[ii][jj]=y_ox_box[ii][jj]-ybox
                ijk = 1
            if y_ox_box[ii][jj]<(-ybox/2):
                y_ox_box[ii][jj]=y_ox_box[ii][jj]+ybox
                ijk = 1
            ii=ii+1
        jj=jj+1
        
    ii=0
    jj=0
    while jj < nstep:
        ii=0
        while ii < nhy:
            if x_hy_box[ii][jj]>(xbox/2):
                x_hy_box[ii][jj]=x_hy_box[ii][jj]-xbox
                ijk = 1
            if x_hy_box[ii][jj]<(-xbox/2):
                x_hy_box[ii][jj]=x_hy_box[ii][jj]+xbox
                ijk = 1
            
            if y_hy_box[ii][jj]>(ybox/2):
                y_hy_box[ii][jj]=y_hy_box[ii][jj]-ybox
                ijk = 1
            if y_hy_box[ii][jj]<(-ybox/2):
                y_hy_box[ii][jj]=y_hy_box[ii][jj]+ybox
                ijk = 1
            ii=ii+1
        jj=jj+1
    
    
#print(x_ox[5][1000],x_ox_box[5][1000])

1
1


In [22]:
xperiodic_ox = np.append(x_ox_box,x_ox_box,axis=0) # original and 1
xperiodic_ox = np.append(xperiodic_ox,(x_ox_box+xbox),axis=0) #2
xperiodic_ox = np.append(xperiodic_ox,(x_ox_box+xbox),axis=0) #3
xperiodic_ox = np.append(xperiodic_ox,(x_ox_box+xbox),axis=0) #4
xperiodic_ox = np.append(xperiodic_ox,x_ox_box,axis=0)  #5
xperiodic_ox = np.append(xperiodic_ox,(x_ox_box-xbox),axis=0) #6
xperiodic_ox = np.append(xperiodic_ox,(x_ox_box-xbox),axis=0) #7
xperiodic_ox = np.append(xperiodic_ox,(x_ox_box-xbox),axis=0) #8

yperiodic_ox = np.append(y_ox_box,(y_ox_box+ybox),axis=0) # original and 1
yperiodic_ox = np.append(yperiodic_ox,(y_ox_box+ybox),axis=0) #2
yperiodic_ox = np.append(yperiodic_ox,(y_ox_box),axis=0) #3
yperiodic_ox = np.append(yperiodic_ox,(y_ox_box-ybox),axis=0) #4
yperiodic_ox = np.append(yperiodic_ox,(y_ox_box-ybox),axis=0) #5
yperiodic_ox = np.append(yperiodic_ox,(y_ox_box-ybox),axis=0) #6
yperiodic_ox = np.append(yperiodic_ox,(y_ox_box),axis=0) #7
yperiodic_ox = np.append(yperiodic_ox,(y_ox_box+ybox),axis=0) #8

zperiodic_ox = (z_ox_box,z_ox_box,z_ox_box,z_ox_box,z_ox_box,z_ox_box,z_ox_box,z_ox_box,z_ox_box)
zperiodic_ox = np.concatenate(zperiodic_ox)

periodic_ox=(ox,ox,ox,ox,ox,ox,ox,ox,ox)
periodic_ox=np.concatenate(periodic_ox)

In [23]:
def distans(xo,yo,zo,xh,yh,zh):
    distan = np.sqrt((xh-xo)**2+(yh-yo)**2+(zh-zo)**2)
    #print(distan)
    return distan

In [24]:
def water(xo,yo,zo,xh,yh,zh,hy,periodic_ox,nox,nhy):
    jj=0
    rij= np.empty(nhy*nox*9).reshape(nhy,nox*9)
    while jj < nhy:
        ii=0
        while ii < (nox*9):
            rij[jj][ii]=distans(xo[ii],yo[ii],zo[ii],xh[jj],yh[jj],zh[jj])
            ii = ii+1
        jj = jj+1
        
    hoatm = np.empty(nhy*2,dtype=int).reshape(nhy,2)
    wht = np.empty(nox*6,dtype=int).reshape(nox,6)
    hoatm[:][:]=0
    wht[:][:]=0
    j=0 
    while j<nhy:#go over H 
        hh = rij[j][0]
        hoatm[j][0] = hy[j]
        hoatm[j][1] = periodic_ox[0]
        i=1
        while i<(nox*9): #go over in periodic box
            if rij[j][i]<hh: #find shotest bond
                hoatm[j][1] = periodic_ox[i]
                hh = rij[j][i]
            i=i+1
        j=j+1
    nn=0
    while nn<nox:
        wht[nn][0]= periodic_ox[nn]
        mm=0
        kk=1
        while mm<nhy:
            if hoatm[mm][1]==periodic_ox[nn]:
                wht[nn][kk] = hoatm[mm][0]
                kk=kk+1
            mm=mm+1
        nn=nn+1
    return wht

In [26]:
f=open('hoping_H.dat','w')
ll=0
while ll<nstep:
    xo = [sub[ll] for sub in xperiodic_ox]
    yo = [sub[ll] for sub in yperiodic_ox]
    zo = [sub[ll] for sub in zperiodic_ox]
    xh = [sub[ll] for sub in x_hy_box]
    yh = [sub[ll] for sub in y_hy_box]
    zh = [sub[ll] for sub in z_hy_box]
    wht = water(xo,yo,zo,xh,yh,zh,hy,periodic_ox,nox,nhy)
    
    
    #print(wht)
    findH = np.where(wht == clos_O_index[ll])
    index = findH[0][0]
    
    ik, = np.where(hy==wht[index][1])
    
    findOH, = np.where(ox == oh_index[ll])
    
    closO, = np.where(ox == clos_O_index[ll])
    #print(findOH)
    
    #print(index,clos_O_index[ll])
    #print(wht[index][1],wht[index][2])
    #print(ik[0])
    #print(xv_hy[ik[0]][ll],x_hy_box[ik[0]][ll])
    
    dis1 = distans(x_ox_box[findOH[0]][ll],y_ox_box[findOH[0]][ll],z_ox_box[findOH[0]][ll],x_hy_box[ik[0]][ll],y_hy_box[ik[0]][ll],z_hy_box[ik[0]][ll])
    #print('diss1 = ', dis1)
    
    ik2, = np.where(hy==wht[index][2])
    dis2 = distans(x_ox_box[findOH[0]][ll],y_ox_box[findOH[0]][ll],z_ox_box[findOH[0]][ll],x_hy_box[ik2[0]][ll],y_hy_box[ik2[0]][ll],z_hy_box[ik2[0]][ll])
    #print('diss2 = ', dis2)
    
    if (dis1<dis2):
        hopH = ik[0]
    else:
        hopH = ik2[0]
    #print('hopH = ', hopH, hy[hopH])
    

      
    print('{:<8d}'.format(ll),'{:<5d}'.format(hy[hopH]),'{0: >#016.8f}'.format(x_hy_box[hopH][ll]),'{0: >#016.8f}'.format(y_hy_box[hopH][ll]),'{0: >#016.8f}'.format(z_hy_box[hopH][ll]),'{0: >#016.8f}'.format(xv_hy[hopH][ll]),'{0: >#016.8f}'.format(yv_hy[hopH][ll]),'{0: >#016.8f}'.format(zv_hy[hopH][ll]),'   ','{:<5d}'.format(oh_index[ll]),'{0: >#016.8f}'.format(x_ox_box[findOH[0]][ll]),'{0: >#016.8f}'.format(y_ox_box[findOH[0]][ll]),'{0: >#016.8f}'.format(z_ox_box[findOH[0]][ll]),'{0: >#016.8f}'.format(xv_ox[findOH[0]][ll]),'{0: >#016.8f}'.format(yv_ox[findOH[0]][ll]),'{0: >#016.8f}'.format(zv_ox[findOH[0]][ll]),'   ','{:<5d}'.format(clos_O_index[ll]),'{0: >#016.8f}'.format(x_ox_box[closO[0]][ll]),'{0: >#016.8f}'.format(y_ox_box[closO[0]][ll]),'{0: >#016.8f}'.format(z_ox_box[closO[0]][ll]),'{0: >#016.8f}'.format(xv_ox[closO[0]][ll]),'{0: >#016.8f}'.format(yv_ox[closO[0]][ll]),'{0: >#016.8f}'.format(zv_ox[closO[0]][ll]),file =f)
    #print('{:<6d}'.format(ll),'{:<6d}'.format(ioh),'{0: >#016.8f}'.format(x_ox[joh][ll]),'{0: >#016.8f}'.format(y_ox[joh][ll]),'{0: >#016.8f}'.format(z_ox[joh][ll]),file =f )
    ll=ll+1
    
f.close()

In [27]:
ll=0
while ll<nstep:
    ii, = np.where(ox==oh_index[ll])
    print('{:<6d}'.format(ll),'{:<6d}'.format(int(oh_index[ll])),'{0: >#016.8f}'.format(xv_ox[ii[0]][ll]),'{0: >#016.8f}'.format(yv_ox[ii[0]][ll]),'{0: >#016.8f}'.format(zv_ox[ii[0]][ll]))  
    ll = ll+1

0      301          4.36608097       1.73684537      -2.16551324
1      301          5.93645082       2.49080375      -0.30018404
2      301          4.80951535       0.20061263      -0.04143303
3      301          0.11534664      -2.11477197      -3.01993810
4      301         -1.59919607      -1.73970790      -4.45768637
5      301         -1.65535553       1.53407386      -2.46250628
6      301         -4.30775767       2.95705789      -0.15114566
7      280         -3.12151578       3.47841000       3.40279136
8      301         -3.68208587       3.79712161       1.52891882
9      280         -6.75259104       2.02663330       0.11104204
10     301          1.82240833       2.18366362       2.01795367
11     301          3.46192409      -1.04096433       0.93438986
12     301          0.13819535      -2.86092068      -1.50942849
13     301         -1.16138149      -1.27939806      -3.65553439
14     301         -2.03902560       2.48206866      -4.83015302
15     301         -2.481

666    289          3.40179128       0.87359674       3.73074428
667    289          0.22820530      -1.34359436       4.64350719
668    289         -2.06527506      -4.38200441       7.06262046
669    289         -2.17833861      -6.45186935       8.77458508
670    289         -1.95098935      -6.27332434       9.83862903
671    289         -1.65679615      -0.43601004       8.72816543
672    289          1.06399397       6.69053873       5.10938738
673    289          2.78459825      11.23057252       1.44915455
674    289          5.69460069       8.83947933       0.81698336
675    289          2.98910650       4.52373937      -0.16860014
676    289         -0.34222469       3.92670055      -1.69614895
677    289         -3.56408531       5.78233692      -1.66837123
678    289         -3.04638891       5.18840657      -1.10946341
679    289         -2.16804676       3.70228774       0.34117866
680    289         -1.37901496       2.85933064       0.15407402
681    289         -0.389